##  8. Закрепление знаний

 Настало время потренироваться в очистке данных! В этот раз тема особенно важная — поговорим о диабете.

Вам предоставлен набор данных, первоначально полученный в Национальном институте диабета, болезней органов пищеварения и почек. 

Этот набор данных создан для того, чтобы на основе определённых диагностических измерений предсказать, есть ли у пациента диабет. 

На выбор этих экземпляров из более крупной базы данных было наложено несколько ограничений. В частности, все пациенты здесь — женщины не моложе 21 года индейского происхождения Пима.

Прочитаем наши данные и выведем первые пять строк таблицы:

In [1]:
import pandas as pd
from pandas import DataFrame
import numpy as np
from typing import Union

diabetes = pd.read_csv('data/diabetes_data.csv')
diabetes.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,Gender
0,6,98,58,33,190,34.0,0.430,43,0,Female
1,2,112,75,32,0,35.7,0.148,21,0,Female
2,2,108,64,0,0,30.8,0.158,21,0,Female
3,8,107,80,0,0,24.6,0.856,34,0,Female
4,7,136,90,0,0,29.9,0.210,50,0,Female


### Признаки в данных

Pregnancies — количество беременностей.

Glucose — концентрация глюкозы в плазме через два часа при пероральном тесте на толерантность к глюкозе.

BloodPressure — диастолическое артериальное давление (мм рт. ст.).

SkinThickness — толщина кожной складки трицепса (мм).

Insulin — двухчасовой сывороточный инсулин (ме Ед/мл).

BMI — индекс массы тела ( (вес в кг)/ (рост в м)**2)

DiabetesPedigreeFunction — функция родословной диабета (чем она выше, тем выше шанс наследственной заболеваемости).

Age — возраст.

Outcome — наличие диабета (0 — нет, 1 — да).

Предварительно вы можете провести небольшой разведывательный анализ: посмотреть на распределения признаков и оценить их взаимосвязь с признаком наличия диабета.

### Задание 8.1
1 point possible (graded)
Начнём с поиска дубликатов в данных. Найдите все повторяющиеся строки в данных и удалите их. Для поиска используйте все признаки в данных. Сколько записей осталось в данных?

In [2]:
dupl_columns = list(diabetes.columns)
#dupl_columns.remove('id')

mask = diabetes.duplicated(subset=dupl_columns)
diabetes_duplicates = diabetes[mask]
print(f'Число найденных дубликатов: {diabetes.shape[0]}')
diabetes_dedupped = diabetes.drop_duplicates(subset=dupl_columns)
print(f'Результирующее число записей: {diabetes_dedupped.shape[0]}')

Число найденных дубликатов: 778
Результирующее число записей: 768


### Задание 8.2
1 point possible (graded)
Далее найдите все неинформативные признаки в данных и избавьтесь от них. В качестве порога информативности возьмите 0.95: удалите все признаки, для которых 95 % значений повторяются или 95 % записей уникальны. В ответ запишите имена признаков, которые вы нашли (без кавычек).

Ответ:
Gender

In [3]:
#список неинформативных признаков
low_information_cols = [] 

#цикл по всем столбцам
for col in diabetes_dedupped.columns:
    #наибольшая относительная частота в признаке
    top_freq = diabetes_dedupped[col].value_counts(normalize=True).max()
    #доля уникальных значений от размера признака
    nunique_ratio = diabetes_dedupped[col].nunique() / diabetes_dedupped[col].count()
    # сравниваем наибольшую частоту с порогом
    if top_freq > 0.95:
        low_information_cols.append(col)
        print(f'{col}: {round(top_freq*100, 2)}% одинаковых значений')
    # сравниваем долю уникальных значений с порогом
    if nunique_ratio > 0.95:
        low_information_cols.append(col)
        print(f'{col}: {round(nunique_ratio*100, 2)}% уникальных значений')

print(f'Неинформативные признаки: {low_information_cols}')
print(f'Количество неинформативных признаков: {len(low_information_cols)}')


information_diabetes_dedupped = diabetes_dedupped.drop(low_information_cols, axis=1)
print(f'Результирующее число признаков: {information_diabetes_dedupped.shape[1]}')


Gender: 100.0% одинаковых значений
Неинформативные признаки: ['Gender']
Количество неинформативных признаков: 1
Результирующее число признаков: 9


### Задание 8.3
1 point possible (graded)
Попробуйте найти пропуски в данных с помощью метода isnull().

Спойлер: ничего не найдёте. А они есть! Просто они скрыты от наших глаз. В таблице пропуски в столбцах Glucose, BloodPressure, SkinThickness, Insulin и BMI обозначены нулём, поэтому традиционные методы поиска пропусков ничего вам не покажут. Давайте это исправим!

Замените все записи, равные 0, в столбцах Glucose, BloodPressure, SkinThickness, Insulin и BMI на символ пропуска. Его вы можете взять из библиотеки numpy: np.nan.
Какая доля пропусков содержится в столбце Insulin? Ответ округлите до сотых.

Ответ:


In [13]:
#display(information_diabetes_dedupped.info())

cols_null_percent = information_diabetes_dedupped.isnull().mean() * 100
cols_with_null = cols_null_percent[cols_null_percent>0].sort_values(ascending=False)
print(f"Вариант1. Cтолбцы с пропусками: \n {cols_with_null} \n")


# Копирование датафрейма
diabetes_with_nan = information_diabetes_dedupped.copy()

# Замена 0 на np.nan в определенных столбцах
columns_to_replace = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
diabetes_with_nan[columns_to_replace] = diabetes_with_nan[columns_to_replace].replace(0, np.nan)


cols_null_percent = diabetes_with_nan.isnull().mean() * 100
cols_with_null = cols_null_percent[cols_null_percent>0].sort_values(ascending=False)
#display(cols_with_null)

print(f"Вариант2. Cтолбцы с пропусками: \n {cols_with_null} \n")

# Расчет доли пропорции пустых значений в колонке Insulin
missing_proportion = diabetes_with_nan['Insulin'].isna().mean()

print(f"Доля пропусков содержится в столбце Insulin равна {missing_proportion:.2f}.")



Вариант1. Cтолбцы с пропусками: 
 Series([], dtype: float64) 

Вариант2. Cтолбцы с пропусками: 
 Insulin          48.697917
SkinThickness    29.557292
BloodPressure     4.557292
BMI               1.432292
Glucose           0.651042
dtype: float64 

Доля пропусков содержится в столбце Insulin равна 0.49.


### Задание 8.4
1 point possible (graded)
Удалите из данных признаки, где число пропусков составляет более 30 %. Сколько признаков осталось в ваших данных (с учетом удаленных неинформативных признаков в задании 8.2)?


Ответ:


In [16]:
#создаем копию исходной таблицы
diabetes_drop = diabetes_with_nan.copy()
#задаем минимальный порог: вычисляем 70% от числа строк
thresh = diabetes_drop.shape[0]*0.7
#удаляем столбцы, в которых более 30% (100-70) пропусков
diabetes_drop = diabetes_drop.dropna(thresh=thresh, axis=1)
#удаляем записи, в которых есть хотя бы 1 пропуск
#diabetes_drop = diabetes_drop.dropna(how='any', axis=0)
#отображаем результирующую долю пропусков
diabetes_drop.isnull().mean()

print(diabetes_drop.shape[1])

display(diabetes_drop.info())

8
<class 'pandas.core.frame.DataFrame'>
Index: 768 entries, 0 to 767
Data columns (total 8 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   763 non-null    float64
 2   BloodPressure             733 non-null    float64
 3   SkinThickness             541 non-null    float64
 4   BMI                       757 non-null    float64
 5   DiabetesPedigreeFunction  768 non-null    float64
 6   Age                       768 non-null    int64  
 7   Outcome                   768 non-null    int64  
dtypes: float64(5), int64(3)
memory usage: 54.0 KB


None

### Задание 8.5
1 point possible (graded)
Удалите из данных только те строки, в которых содержится более двух пропусков одновременно. Чему равно результирующее число записей в таблице?

Ответ:


In [18]:

print(f"Число записей в таблице до удаления NaN = {diabetes_drop.shape[0]}.")

# Удалить строки с более чем 2 NaN значениями

diabetes_cleaned = diabetes_drop[diabetes_drop.isnull().sum(axis=1) <= 2]

# Результирующее число записей в таблице
resulting_records = diabetes_cleaned.shape[0]

print(f"Результирующее число записей в таблице = {resulting_records}.")

Число записей в таблице до удаления NaN = 768.
Результирующее число записей в таблице = 761.


### Задание 8.6
1 point possible (graded)
В оставшихся записях замените пропуски на медиану. Чему равно среднее значение в столбце SkinThickness? Ответ округлите до десятых.

Ответ:

In [23]:
#создаем копию исходной таблицы
diabetes_filled = diabetes_cleaned.copy()
#создаем словарь имя столбца: число(признак) на который надо заменить пропуски
# values = {
#     'Pregnancies': diabetes_filled['Pregnancies'].median(),
#     'Glucose': diabetes_filled['Glucose'].median(),
#     'BloodPressure': diabetes_filled['BloodPressure'].median(),
#     'SkinThickness': diabetes_filled['SkinThickness'].median(),
#     'BMI': diabetes_filled['BMI'].median(),
#     'DiabetesPedigreeFunction': diabetes_filled['DiabetesPedigreeFunction'].median(),
#     'Age': diabetes_filled['Age'].median(),
#     'Outcome': diabetes_filled['Outcome'].median()
# }


#заполняем пропуски в соответствии с заявленным словарем
#diabetes_filled = diabetes_filled.fillna(values)
diabetes_filled.fillna(diabetes_filled.median(), inplace=True)

#выводим результирующую долю пропусков
#diabetes_filled.isnull().mean()

mean_skin_thickness = diabetes_filled['SkinThickness'].mean()
print(f"Результирующее среднее значение в столбце SkinThickness = {mean_skin_thickness:.1f}")



Результирующее среднее значение в столбце SkinThickness = 29.1


### Задание 8.7
1 point possible (graded)
Сколько выбросов найдёт классический метод межквартильного размаха в признаке SkinThickness?

Ответ:
87

In [24]:
def outliers_iqr_mod(data: Union[DataFrame, str], feature: str, log_scale=False, left = 1.5, right = 1.5):
    # Проверка, является ли data строкой (именем файла)
    if isinstance(data, str):
        data = pd.read_csv(data)
    if log_scale:
        #x = np.log(data[feature]+1)
        x = np.log(data[feature]) # убран +1 т.к. в признаке нет нулей
    else:
        x = data[feature]    
    quartile_1, quartile_3 = x.quantile(0.25), x.quantile(0.75),
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * left)
    upper_bound = quartile_3 + (iqr * right)
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x >= lower_bound) & (x <= upper_bound)]
    return outliers, cleaned


left_bound = 1.5
right_bound = 1.5
feature = 'SkinThickness'

outliers, cleaned = outliers_iqr_mod(diabetes_filled, feature, False, left_bound, right_bound)
print(f'Число выбросов по методу Тьюки: {outliers.shape[0]}')
print(f'Результирующее число записей: {cleaned.shape[0]}')

Число выбросов по методу Тьюки: 87
Результирующее число записей: 674


### Задание 8.8
1 point possible (graded)
Сколько выбросов найдёт классический метод z-отклонения в признаке SkinThickness?

Ответ:
4

In [26]:
def outliers_z_score_mod(data: Union[DataFrame, str], feature, log_scale=False, left=3, right=3):
    # Проверка, является ли data строкой (именем файла)
    if isinstance(data, str):
        data = pd.read_csv(data)    
    if log_scale:
        x = np.log(data[feature]+1)
    else:
        x = data[feature]
    mu = x.mean()
    sigma = x.std()
    lower_bound = mu - left * sigma
    upper_bound = mu + right * sigma
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x >= lower_bound) & (x <= upper_bound)]
    return outliers, cleaned

left_bound = 3.0
right_bound = 3.0
log_scale=False
feature = 'SkinThickness'

outliers, cleaned = outliers_z_score_mod(diabetes_filled, feature, log_scale, left_bound, right_bound)
print(f'Число выбросов по методу z-отклонения: {outliers.shape[0]}')
print(f'Результирующее число записей: {cleaned.shape[0]}')


Число выбросов по методу z-отклонения: 4
Результирующее число записей: 757


### Задание 8.9
1 point possible (graded)
На приведённой гистограмме показано распределение признака DiabetesPedigreeFunction. Такой вид распределения очень похож на логнормальный, и он заставляет задуматься о логарифмировании признака. Найдите сначала число выбросов в признаке DiabetesPedigreeFunction с помощью классического метода межквартильного размаха.

Затем найдите число выбросов в этом же признаке в логарифмическом масштабе (при логарифмировании единицу прибавлять не нужно!). Какова разница между двумя этими числами (вычтите из первого второе)?

Ответ:




In [28]:
def outliers_z_score_mod(data: Union[DataFrame, str], feature, log_scale=False, left=3, right=3):
    # Проверка, является ли data строкой (именем файла)
    if isinstance(data, str):
        data = pd.read_csv(data)    
    if log_scale:
#        x = np.log(data[feature]+1) # убран +1 т.к. в признаке нет нулей
        x = np.log(data[feature])

    else:
        x = data[feature]
    mu = x.mean()
    sigma = x.std()
    lower_bound = mu - left * sigma
    upper_bound = mu + right * sigma
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x >= lower_bound) & (x <= upper_bound)]
    return outliers, cleaned

left_bound = 1.5
right_bound = 1.5
log_scale=False
feature = 'DiabetesPedigreeFunction'

outliers, cleaned = outliers_iqr_mod(diabetes_filled, feature, log_scale, left_bound, right_bound)
print(f'Число выбросов по методу Тьюки (log_scale = {log_scale}): {outliers.shape[0]}')
print(f'Результирующее число записей (log_scale = {log_scale}): {cleaned.shape[0]}')
outliers_log_false = outliers.shape[0]


left_bound = 1.5
right_bound = 1.5
log_scale=True
feature = 'DiabetesPedigreeFunction'

outliers, cleaned = outliers_iqr_mod(diabetes_filled, feature, log_scale, left_bound, right_bound)
print(f'Число выбросов по методу Тьюки (log_scale = {log_scale}): {outliers.shape[0]}')
print(f'Результирующее число записей (log_scale = {log_scale}): {cleaned.shape[0]}')
outliers_log_true = outliers.shape[0]

print(f'Разница числа выбросов при log_scale = False и log_scale = True: {outliers_log_false - outliers_log_true}')

Число выбросов по методу Тьюки (log_scale = False): 29
Результирующее число записей (log_scale = False): 732
Число выбросов по методу Тьюки (log_scale = True): 0
Результирующее число записей (log_scale = True): 761
Разница числа выбросов при log_scale = False и log_scale = True: 29
